In [1]:
# Import Libraries
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

In [3]:
# Scrape shots data from Understats
# MD3: BVB vs Werder Bremen 8/20/22
understats_url = 'https://understat.com/match/19364'

# Parse Understats' webpage
res = requests.get(understats_url)
soup = BeautifulSoup(res.content, 'lxml')
scripts = soup.find_all('script')

# View
scripts

[<script>
 			var THEME = localStorage.getItem("theme") || 'DARK';
 			document.body.className = "theme-" + THEME.toLowerCase();
 		</script>,
 <script>
 	var shotsData 	= JSON.parse('\x7B\x22h\x22\x3A\x5B\x7B\x22id\x22\x3A\x22481700\x22,\x22minute\x22\x3A\x221\x22,\x22result\x22\x3A\x22MissedShots\x22,\x22X\x22\x3A\x220.72\x22,\x22Y\x22\x3A\x220.5690000152587891\x22,\x22xG\x22\x3A\x220.015994248911738396\x22,\x22player\x22\x3A\x22Mahmoud\x20Dahoud\x22,\x22h_a\x22\x3A\x22h\x22,\x22player_id\x22\x3A\x22205\x22,\x22situation\x22\x3A\x22OpenPlay\x22,\x22season\x22\x3A\x222022\x22,\x22shotType\x22\x3A\x22RightFoot\x22,\x22match_id\x22\x3A\x2219364\x22,\x22h_team\x22\x3A\x22Borussia\x20Dortmund\x22,\x22a_team\x22\x3A\x22Werder\x20Bremen\x22,\x22h_goals\x22\x3A\x222\x22,\x22a_goals\x22\x3A\x223\x22,\x22date\x22\x3A\x222022\x2D08\x2D20\x2013\x3A30\x3A00\x22,\x22player_assisted\x22\x3A\x22Jamie\x20Bynoe\x2DGittens\x22,\x22lastAction\x22\x3A\x22Pass\x22\x7D,\x7B\x22id\x22\x3A\x22481707\x22,\x22

In [4]:
# Grab shotsData
shots_strings = scripts[1].string

# View
shots_strings

"\n\tvar shotsData \t= JSON.parse('\\x7B\\x22h\\x22\\x3A\\x5B\\x7B\\x22id\\x22\\x3A\\x22481700\\x22,\\x22minute\\x22\\x3A\\x221\\x22,\\x22result\\x22\\x3A\\x22MissedShots\\x22,\\x22X\\x22\\x3A\\x220.72\\x22,\\x22Y\\x22\\x3A\\x220.5690000152587891\\x22,\\x22xG\\x22\\x3A\\x220.015994248911738396\\x22,\\x22player\\x22\\x3A\\x22Mahmoud\\x20Dahoud\\x22,\\x22h_a\\x22\\x3A\\x22h\\x22,\\x22player_id\\x22\\x3A\\x22205\\x22,\\x22situation\\x22\\x3A\\x22OpenPlay\\x22,\\x22season\\x22\\x3A\\x222022\\x22,\\x22shotType\\x22\\x3A\\x22RightFoot\\x22,\\x22match_id\\x22\\x3A\\x2219364\\x22,\\x22h_team\\x22\\x3A\\x22Borussia\\x20Dortmund\\x22,\\x22a_team\\x22\\x3A\\x22Werder\\x20Bremen\\x22,\\x22h_goals\\x22\\x3A\\x222\\x22,\\x22a_goals\\x22\\x3A\\x223\\x22,\\x22date\\x22\\x3A\\x222022\\x2D08\\x2D20\\x2013\\x3A30\\x3A00\\x22,\\x22player_assisted\\x22\\x3A\\x22Jamie\\x20Bynoe\\x2DGittens\\x22,\\x22lastAction\\x22\\x3A\\x22Pass\\x22\\x7D,\\x7B\\x22id\\x22\\x3A\\x22481707\\x22,\\x22minute\\x22\\x3A\\x2246\\

In [5]:
# Strip unnecessary symbols and get only JSON data
ind_start = shots_strings.index("('")+2
ind_end = shots_strings.index("')")
json_data = shots_strings[ind_start:ind_end]

# Get JSON data
json_data = json_data.encode('utf8').decode('unicode_escape')

# Convert to JSON format
shots_data = json.loads(json_data)

# View
shots_data

{'h': [{'id': '481700',
   'minute': '1',
   'result': 'MissedShots',
   'X': '0.72',
   'Y': '0.5690000152587891',
   'xG': '0.015994248911738396',
   'player': 'Mahmoud Dahoud',
   'h_a': 'h',
   'player_id': '205',
   'situation': 'OpenPlay',
   'season': '2022',
   'shotType': 'RightFoot',
   'match_id': '19364',
   'h_team': 'Borussia Dortmund',
   'a_team': 'Werder Bremen',
   'h_goals': '2',
   'a_goals': '3',
   'date': '2022-08-20 13:30:00',
   'player_assisted': 'Jamie Bynoe-Gittens',
   'lastAction': 'Pass'},
  {'id': '481707',
   'minute': '46',
   'result': 'Goal',
   'X': '0.8190000152587891',
   'Y': '0.3120000076293945',
   'xG': '0.030886465683579445',
   'player': 'Julian Brandt',
   'h_a': 'h',
   'player_id': '261',
   'situation': 'OpenPlay',
   'season': '2022',
   'shotType': 'LeftFoot',
   'match_id': '19364',
   'h_team': 'Borussia Dortmund',
   'a_team': 'Werder Bremen',
   'h_goals': '2',
   'a_goals': '3',
   'date': '2022-08-20 13:30:00',
   'player_assiste

In [19]:
# Collect data

# Initialize empty list
minute = []
result = []
xg = []
player = []
situation = []
shot_type = []
club = []

# Select home & away
data_home = shots_data['h']
data_away = shots_data['a']

In [20]:
# Append home team data to the lists
for index in range(len(data_home)):
    for key in data_home[index]:
        if key == 'minute':
            minute.append(data_home[index][key])
        if key == 'result':
            result.append(data_home[index][key])
        if key == 'xG':
            xg.append(data_home[index][key])
        if key == 'player':
            player.append(data_home[index][key])
        if key == 'situation':
            situation.append(data_home[index][key])
        if key == 'shotType':
            shot_type.append(data_home[index][key])
        if key == 'h_team':
            club.append(data_home[index][key])
            
# Append home team data
for index in range(len(data_away)):
    for key in data_away[index]:
        if key == 'minute':
            minute.append(data_away[index][key])
        if key == 'result':
            result.append(data_away[index][key])
        if key == 'xG':
            xg.append(data_away[index][key])
        if key == 'player':
            player.append(data_away[index][key])
        if key == 'situation':
            situation.append(data_away[index][key])
        if key == 'shotType':
            shot_type.append(data_away[index][key])
        if key == 'a_team':
            club.append(data_away[index][key])

In [22]:
# Create df
col_names = ['minute', 'result', 'xG', 'player', 'situation', 'shot_type', 'club']
shot_df = pd.DataFrame([minute, result, xg, player, situation, shot_type, club],
                       index = col_names)
shot_df = shot_df.T

# View df
shot_df

,minute,result,xG,player,situation,shot_type,club
0,1,MissedShots,0.015994248911738396,Mahmoud Dahoud,OpenPlay,RightFoot,Borussia Dortmund
1,46,Goal,0.030886465683579445,Julian Brandt,OpenPlay,LeftFoot,Borussia Dortmund
2,65,BlockedShot,0.09414775669574738,Jude Bellingham,OpenPlay,RightFoot,Borussia Dortmund
3,76,Goal,0.018221033737063408,Raphael Guerreiro,FromCorner,LeftFoot,Borussia Dortmund
4,79,SavedShot,0.030797746032476425,Thorgan Hazard,OpenPlay,RightFoot,Borussia Dortmund
5,89,SavedShot,0.4354998767375946,Youssoufa Moukoko,OpenPlay,LeftFoot,Borussia Dortmund
6,3,BlockedShot,0.011031766422092915,Niclas Füllkrug,OpenPlay,RightFoot,Werder Bremen
7,20,MissedShots,0.43170279264450073,Jens Stage,SetPiece,Head,Werder Bremen
8,32,BlockedShot,0.013912087306380272,Christian Groß,FromCorner,RightFoot,Werder Bremen
9,33,SavedShot,0.08418288826942444,Marvin Ducksch,OpenPlay,RightFoot,Werder Bremen


In [24]:
# Save csv file
shot_df.to_csv('./dataset/MD3_BVB_WerderBremen_08222022.csv', index = False)